Name : Vani Balani

Roll No : 21BCE021

Course & Course Code : 2CSDE93

Practical No : 2

AIM : To create a blockchain and implement replay attacks on blockchain


In [ ]:
import hashlib
import datetime

In [11]:
class Block:
    def __init__(self, index, previous_hash, timestamp, transaction_data, current_hash, nonce):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.transaction_data = transaction_data
        self.current_hash = current_hash
        self.nonce = nonce

    def __repr__(self):
        return f"Block(Index: {self.index}, Hash: {self.current_hash}, Previous Hash: {self.previous_hash}, Timestamp: {self.timestamp}, Data: {self.transaction_data}, Nonce: {self.nonce})"

class Blockchain:
    def __init__(self, difficulty_level=2):
        self.chain = [self.generate_genesis_block()]
        self.difficulty_level = difficulty_level

    def generate_genesis_block(self):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return Block(0, "0", timestamp, "Initial Block", self.compute_block_hash(0, "0", timestamp, "Initial Block", 0), 0)

    def retrieve_last_block(self):
        return self.chain[-1]

    def append_block(self, transaction_data):
        last_block = self.retrieve_last_block()
        new_index = last_block.index + 1
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        previous_hash = last_block.current_hash
        nonce, current_hash = self.compute_proof_of_work(new_index, previous_hash, timestamp, transaction_data)
        new_block = Block(new_index, previous_hash, timestamp, transaction_data, current_hash, nonce)
        self.chain.append(new_block)

    def compute_block_hash(self, index, previous_hash, timestamp, transaction_data, nonce):
        value = f"{index}{previous_hash}{timestamp}{transaction_data}{nonce}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    def compute_proof_of_work(self, index, previous_hash, timestamp, transaction_data):
        nonce = 0
        while True:
            current_hash = self.compute_block_hash(index, previous_hash, timestamp, transaction_data, nonce)
            if current_hash[:self.difficulty_level] == '0' * self.difficulty_level:
                return nonce, current_hash
            nonce += 1

    def validate_chain(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            recalculated_hash = self.compute_block_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.transaction_data, current_block.nonce)
            if current_block.current_hash != recalculated_hash:
                return False
            if current_block.previous_hash != previous_block.current_hash:
                return False
        return True

    def alter_block(self, index, updated_data):
        if 0 < index < len(self.chain):
            self.chain[index].transaction_data = updated_data
            self.update_block_hash(index)

    def update_block_hash(self, index):
        current_block = self.chain[index]
        previous_hash = self.chain[index-1].current_hash if index > 0 else "0"
        nonce, current_hash = self.compute_proof_of_work(current_block.index, previous_hash, current_block.timestamp, current_block.transaction_data)
        current_block.current_hash = current_hash
        current_block.nonce = nonce
        current_block.previous_hash = previous_hash

In [12]:
def run_blockchain_operations():
    blockchain = Blockchain(difficulty_level=2)
    blockchain.append_block("Transaction 1: Alice pays Bob 10 BTC")
    blockchain.append_block("Transaction 2: Bob pays Charlie 5 BTC")
    blockchain.append_block("Transaction 3: Charlie pays Alice 2 BTC")

    print("Initial Blockchain:")
    for block in blockchain.chain:
        print(block)

    print("\nBlockchain validity -> ", blockchain.validate_chain())

    blockchain.alter_block(1, "Tampered Transaction: Alice pays Bob 100 BTC")
    print("\nBlockchain post-tampering:")
    for block in blockchain.chain:
        print(block)

    print("\nBlockchain validity -> ", blockchain.validate_chain())

run_blockchain_operations()

Initial Blockchain:
Block(Index: 0, Hash: e5ea0c8debb8de532ae21b871ae7bfe658e98208e2563de21fa67d841d7abda3, Previous Hash: 0, Timestamp: 2024-08-10 18:13:10, Data: Initial Block, Nonce: 0)
Block(Index: 1, Hash: 00f0dca583caf007528f7f110b571429e6da84c64d49122389981cffb85e87e2, Previous Hash: e5ea0c8debb8de532ae21b871ae7bfe658e98208e2563de21fa67d841d7abda3, Timestamp: 2024-08-10 18:13:10, Data: Transaction 1: Alice pays Bob 10 BTC, Nonce: 335)
Block(Index: 2, Hash: 00e0126aded79014d27f26c0c7ea5d037cb1cd1b69fd3eacad232a1e38526fa1, Previous Hash: 00f0dca583caf007528f7f110b571429e6da84c64d49122389981cffb85e87e2, Timestamp: 2024-08-10 18:13:10, Data: Transaction 2: Bob pays Charlie 5 BTC, Nonce: 98)
Block(Index: 3, Hash: 009c5ddeec16ad100cc1211ed41640db71cfb08312c416d19c66f41fa1f6707c, Previous Hash: 00e0126aded79014d27f26c0c7ea5d037cb1cd1b69fd3eacad232a1e38526fa1, Timestamp: 2024-08-10 18:13:10, Data: Transaction 3: Charlie pays Alice 2 BTC, Nonce: 104)

Blockchain validity ->  True

Blockc